Arquivo de simulação utilizando sinais gerados utilizando o módulo Numpy.

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display as ld
import musdb
from IPython.display import Audio
from scipy import stats
import soundfile as sf
import pandas as pd
import os
import mir_eval
from scipy.stats import gennorm

In [3]:
def data_dict(tipo, mistura,algoritmo,nfontes,A, W,rate = None, white= None, amari = None, correlacao= None, 
    corr_media = None, mses= None, mse_media = None): 
    data_dict = {'Tipo':tipo, 'Mistura': mistura,'Algoritmo': algoritmo, 'Número de Fontes':nfontes,
                 'Matriz de mistura': A,'Matriz de separacao':W,
                 'Learning Rate':rate, 'White':white,
                 'Indice de Amari': amari,'Correlacao': correlacao,'Correlacao Media': corr_media,
                 'MSE': mses, 'Media EQM': mse_media}
    #data_dict = {'Tipo':tipo, 'Mistura': mistura,'Algoritmo': algoritmo, 'Número de Fontes':nfontes,'Learning Rate':rate, 'White':white,
                 #'Índice de Amari': amari,'Correlacao': correlacao,'MSE': mses, 'SDR': sdr, 'SIR': sir, 'SAR': sar}
    return data_dict

In [4]:
#def source_dict(tipo, mistura, nfontes, s1kurt, s2kurt, s3kurt, s4kurt)

In [5]:
def save_data_csv(data_dict,create = False, path_to_csv = None):
    try:
        df = pd.read_csv(path_to_csv)
        df = df.append(data_dict, ignore_index = True)
        df.to_csv(path_to_csv, index = False)
    except(FileNotFoundError):
        if create:
            df = pd.DataFrame(columns = data_dict.keys())
            df = df.append(data_dict, ignore_index = True)
            df.to_csv(path_to_csv, index = False)
        else:
            raise('DataFrame não encontrado!')
    return df

In [6]:
def whitening(X):
    """Realiza o branqueamento dos dados em X, retornando os dados branqueados em z junto 
    da matriz de branqueamento em Wwhite."""
    
    Xm = X - X.mean()
    covXm = np.cov(Xm)
    evalor, evetor = np.linalg.eig(covXm)
    wevalor = np.diag(1/(evalor**0.5))
    Wwhite = wevalor.dot(evetor.T) #matriz de branqueamento
    z = np.dot(Wwhite,Xm)
    return z, Wwhite

In [7]:
def normaliza(v):
    """Normaliza os dados recebidos."""

    vn = [k/np.std(k) for k in v]
    return np.array(vn)

In [8]:
def correlacao(Sn, Yn):
    """Realiza o cálculo da correlação de Pearson e separa os pares de maior correlação
    entre sinais originais e sinais estimados
    
    Parâmetros
    -----------------------------
    Sn: array
        Sinais originais normalizados
    Yn: array
        Sinais estimados normalizados
    
    Retorna
    -----------------------------
    allarrays: list 
        Lista contendo duas tuplas, a primeira com os pares de maior correlação, e a segunda com o índice
    de cada componente deste par. Exemplo: [(Sinal1, Sinal4), (0,3)]
    """
    
    allarrays = list()
    for p in range (0,len(Sn)): #Para comparar com S[0] e S[1]
        corrnova = 0
        corr = 0
        arrays = list()
        for o in range(0,len(Sn)): #Para comparar com Yn[0] e Yn[1]
            corrnova = stats.pearsonr(Sn[p],Yn[o])
            #print(f'Correlação de Pearson vale {stats.pearsonr(Sn[p], Yn[o])[0]} entre a {p+1} linha de S e a {o+1} linha de Y')
            if np.abs(corrnova[0]) > np.abs(corr): 
                corr = corrnova[0]
                arrays.clear()
                if corr < 0:
                    Yn[o] = Yn[o] * -1
                arrays.append(((Sn[p], Yn[o]),(p,o)))
        allarrays.append(arrays)
    return allarrays 
#Sn[p] e Y[o] são os pares de componentes com maior correlação
#retorna o vetor com os pares de maior correlaçao acompanhado de uma tupla (p,o) em que p é a posiçao de S[p] e o é a posição de Y[o]

In [9]:
def mse(pares, X):
    """Calcula o erro quadrático médio para pares de dados.
    
    Parâmetros
    -----------------------------
    pares: Lista contendo duas tuplas, sendo a primeira um par de variáveis e a segunda o índice dessas variáveis.
    
    Retorna
    -----------------------------
    mseall: Lista com o erro quadrático médio para cada par de dados.
    """
    
    mseall = list()
    msgrafs = list()
    for x in pares:
        mse = 0
        msgraf = list()
        for ponto in range (0, X.shape[1]):
            sqr = (x[0][0][0][ponto]- x[0][0][1][ponto])**2
            msgraf.append(sqr)
            mse += sqr
        msgrafs.append(msgraf)
        mse = mse/X.shape[1]
        posicao = x[0][1] #guarda posiçao das linhas de maior correlaçao
        mseall.append((mse, posicao))
    #for z in mseall:
        #print(f'O erro quadrático médio entre a {(z[1][0])+1}ª linha de S e a {(z[1][1])+1}ª linha de Y é de {z[0]}')
    return mseall, msgrafs
#retorna array com o erro quadratico medio e suas posições correspondentes 

In [10]:
def amari2(W, A):
    """Calcula o índice de amari para W e A """
    p = W.dot(A)
    C= D = 0
    for c in range(len(W)):
        a = b = 0
        for d in range(len(W)):
            a += (np.abs(p[c][d])/np.max(np.abs(p[c,:])))
            b += (np.abs(p[d][c])/np.max(np.abs(p[:,c])))
        C += a-1
        D += b-1
    E = C+D
    print(f'O índice de Amari obtido para W é de {E}')
    return E

In [11]:
def grafsinais(nome, mseall, Sn, Yn, mistura):
    """Plota os gráficos de erro quadrático médio.
    
    Parâmetros
    -----------------------------
    pares: Lista contendo duas tuplas, sendo a primeira um par de variáveis e a segunda o índice dessas variáveis.
    """
    
    for num, a in  enumerate(mseall):
        atual = str(num+1) #indice da fonte atual
        total = str(len(mseall)) #número total de fontes
        indexS = a[1][0]
        indexY = a[1][1]
        plt.figure(figsize = (14,5))
        plt.title(f'{mistura} - Comparação sinais S e Y -{nome} ({atual}/{total})')
        plt.plot(Sn[indexS])
        plt.plot(Yn[indexY])
        plt.legend(['Sinal Original', 'Sinal Separado'], loc = 'lower left')

In [12]:
def grafmse(nome, msegrafs, Sn, Yn, mistura):
    for graf in range (0, len(msegrafs)):
        plt.figure(figsize = (14,5))
        plt.title(f'{mistura} - Erro quadrático S e Y - {nome} ({graf+1}/{len(msegrafs)})')
        #plt.title(f'Erro quadrático médio')
        plt.xlim(0,len(Sn[0]))
        plt.ylim(0,  max(msegrafs[graf]))
        plt.plot(msegrafs[graf], color = 'cornflowerblue')

In [13]:
def curtose(a, w, z, rate= 0, tipo = None):
    """Calcula a matriz de mistura com o algoritmo FastICA com curtose.
    
    Parâmetros
    -----------------------------
    w: Matriz de separação inicial.
    z: Sinal misturado branco.
    
    Retorna
    -----------------------------
    nome: Nome do algoritmo utilizado
    W: Matriz de separação
    """
    
    nome = "FastICA por Curtose"
    wlist = list()
    m = len(z)
    want = 0
    cont = 0
    for p in range (1, m+1):
        w1 = w[:,p-1]
        while True:
            w1 = (((w1.T.dot(z)**3).dot(z.T)).T/z.shape[1])-3*w1
            w1 = w1/np.linalg.norm(w1)
            cont += 1
            if cont >= 2:
                want = w1
                if np.allclose(w1, want, atol = 0.000001):
                    break
            cont += 1  
        if p > 1:
            for j in range(1, p):
                a = w1.T.dot(wlist[j-1])
                w1 = w1 - (a*wlist[j-1])
            w1 = w1/np.linalg.norm(w1)
        wlist.append(w1)
    W = list()
    for o in wlist:
        wfinal = list()
        for numP, p in enumerate(o):
            nw= p
            wfinal.append(nw)
            if numP == len(o)-1:
                W.append(wfinal)            
    W = np.array(W)
    print(f'Número de iterações: {cont}')
    return nome, W, cont

In [14]:
#Algoritmo de FastICA com negentropia
def negentropia(A, w_i, z, rate = 0, tipo = None):
    """Calcula a matriz de separação com o algoritmo FastICA com negentropia.
    
    Parâmetros
    -----------------------------
    w: Matriz de separação inicial.
    z: Sinal misturado branco.
    
    Retorna
    -----------------------------
    nome: Nome do algoritmo utilizado
    W: Matriz de separação
    """
    
    nome = "FastICA por Negentropia"
    wlist = list()
    amaris = list()
    m = len(z)
    want = 0
    cont = 0
    for p in range (1, m+1):
        w = w_i[:,p-1]
        while True:
            w = (z.dot((np.tanh(w.T.dot(z)).T)))/z.shape[1] - (np.sum((1-(np.tanh(w.T.dot(z))**2))/z.shape[1])*w)
            w = w/np.linalg.norm(w)
            cont += 1
            if cont >= 2:
                want = w
                if np.allclose(w, want, atol = 0.000001):
                    break
        if p > 1:
            for j in range(1, p):
                a = w.T.dot(wlist[j-1])
                w = w- (a*wlist[j-1])
            w = w/np.linalg.norm(w)
           # amr = amari2(a,w)
           # amaris.append(amr)
        wlist.append(w)
    W = list()
    for o in wlist:
        wfinal = list()
        for numP, p in enumerate(o):
            nw= p
            wfinal.append(nw)
            if numP == len(o)-1:
                W.append(wfinal)            
    W = np.array(W)
    print(f'Número de iterações: {cont}')
    return nome, W, cont

In [15]:
#Algoritmo Infomax
def infomax(A, W, X, rate, tipo = 'super'):
    """Calcula a matriz de separação com o algoritmo Infomax
    
    Parâmetros
    -----------------------------
    W: array
        Matriz de separação inicial
    X: array
        Mistura de sinais
    rate: float
        Taxa de aprendizagem
 
    tipo : str
        Define qual variação do algoritmo será utilizada (sup/super/extended)
        
    Retorna
    -----------------------------
    nome: str 
        Nome do algoritmo utilizado
    W: array
        Matriz de separação
    """
    
    lenght_X = len(X)
    Wlist = list()
    nome = "Infomax " + tipo
    I = np.eye(lenght_X, lenght_X)
    cont = None
    for k in range (0,X.shape[1]):
        y = W.dot(X[:,k])
        if tipo == 'super':
            fy = np.array([np.tanh(y[m]) for m in range (lenght_X)])
        elif tipo == 'sub':
            fy = np.array([y[m] - np.tanh(y[m]) for m in range (lenght_X)])
        W = W + rate*np.dot((I-np.outer(fy,y)),(W))
        Wlist.append(W)
    #plt.figure()
   # plt.title('Evolução dos pesos - Infomax')
    #for i in range(0, W.shape[0]): #plota a evolução dos pesos
      #  w_pesos = list()
       # for p in Wlist:
         #   w_pesos.append(p[i])
       # plt.plot(w_pesos)
    return nome, W, cont

In [16]:
def correqm_mean(correlacoes_eqms): #media de correlacoes OU eqm
    sum_corr = 0
    for corr_it in range(0, len(correlacoes_eqms)):
        sum_corr+=abs(correlacoes_eqms[corr_it])
    mean_c = sum_corr/len(correlacoes_eqms)
    return mean_c

In [17]:
def teste(tipo,mistura ,S, X, algoritmo, w, A,save = False, rate = 0.001, white = False , tipoinfomax = 'super', path_to_csv = None):
    """Aplica o ICA em uma determinada mistura de acordo com o algoritmo desejado
    
    Parâmetros
    -----------------------------
    S: array 
        Sinais originais
    X: array
        Sinais misturados
    algoritmo: função
        nome da função do algoritmo desejado
    w: array
        matriz de separação inicial
    A: array 
        Matriz de mistura
    rate: float 
        taxa de aprendizagem
    white: Bool
        False caso não se deseje fazer o branqueamento da mistura
        
    Retorna
    -----------------------------
    Yn: array
        Sinais estimados
    
    """
    try:
        Sn = normaliza(S)
        nfontes = len(Sn)
        if white:
            X, wwhite = whitening(X)
        nome, W, steps = algoritmo(A,w, X, rate, tipo = tipoinfomax)
        try:
            y = W.dot(X)
        except(ValueError):
            y = np.transpose(W).dot(X)
        print(f'Nome do algoritmo: {nome}')
        Yn = normaliza(y)
        if white:
            W = W.dot(wwhite)
        amari = amari2(W,A)
        pares = correlacao(Sn,Yn)
        correlacoes = [stats.pearsonr(a[0][0][0], a[0][0][1])[0] for a in pares]
        corr_media = correqm_mean(correlacoes)
        mses, graf_mse = mse(pares, X)
       # grafmse(nome, graf_mse, Sn, Yn, mistura)
       # grafsinais(nome, mses, Sn, Yn, mistura)
        eqm_vect = [eqm[0] for eqm in mses]
        eqm_media = correqm_mean(eqm_vect) 
        A_dict = A.reshape(1,A.shape[0]*A.shape[1])
        W_dict = W.reshape(1, A.shape[0]*W.shape[1])
        A_dict = A_dict[0]
        W_dict = W_dict[0]
        if save == True:
            ddict = data_dict(tipo, mistura,nome,nfontes, A_dict, W_dict,rate, white, amari, correlacoes,corr_media,eqm_vect, eqm_media)
            save_data_csv(ddict, create = True, path_to_csv = path_to_csv)
        print()
        return Yn, pares
    except:
        if save == True:
            ddict = data_dict(tipo, mistura,nome,nfontes, A_dict, W_dict,rate)
            save_data_csv(ddict, create = True, path_to_csv = path_to_csv)
        print('Algoritmo não rodou com sucesso')
        pass

In [18]:
import random

def gen_S_uniform(n_amostras):
    sig_list = list()
    for num in range(1,5):
        sig = np.random.uniform(low = -0.5, high = 0.5, size = n_amostras)
        sig_list.append(sig)
    return sig_list


def gen_S_lap(n_amostras):
    sig_list = list()
    for num in range(1,5):
        sig = np.random.laplace(size = n_amostras)
        sig_list.append(sig)
    return sig_list


def gen_S_norm(n_amostras):
    sig_list = list()
    for num in range(1,5):
        sig = np.random.normal(size = n_amostras)
        sig_list.append(sig)
    return sig_list


def gen_all_gnorm(beta, n_amostras):
    x = np.linspace(gennorm.ppf(0.01, beta),gennorm.ppf(0.99, beta), n_amostras)
    sig = gennorm.pdf(x, beta)
    return sig

def gen_S_gnorm(beta_i, n_amostras):
    sig_list = list()
    for num in range(1,5):
        beta = random.uniform(beta_i, beta_i+1)
        sig = gen_all(beta,n_amostras)
        sig_list.append(sig)
    return sig_list


In [19]:
def simula_gen_nfonts(tipo,n_gen,n_amostra, savewa = False,do_white_fastica = True, do_white_infomax = False,tipoinfomax = 'super', pre_process = True, save = False, path_to_csv = None):
    """Função para realizar as simulações, variando a mistura de fontes e o número de fontes a ser analisado.
        
    Parâmetros
    -----------------------------
    pre_process : bool
        define se o pré processamento deverá ser realizado
    """
    listw_fica = list()
    listw_inf = list()
    for numero in range(1,n_gen):
        mist = 'Mistura '+ str(numero)
        #s1 = np.random.laplace(size = 1000)
        #s2 = np.random.uniform(size = 1000)
        #s3 = np.random.laplace(size=1000)
        if tipo == 'laplace':
            s1, s2, s3, s4  = gen_S_lap(1000)
            tipoinfomax = 'super'
        elif tipo == 'uniform':
            s1, s2, s3, s4  = gen_S_uniform(1000)
            tipoinfomax = 'sub'
        elif tipo == 'normal':
            s1, s2, s3, s4  = gen_S_norm(1000)
        Slist = [s1, s2]
        for n_sources in range(2,5):
            print(f'Número de fontes: {n_sources}')
            S = np.array(Slist)
            lenght_S = len(S)
            A = np.random.randn(lenght_S, lenght_S)
            X = A.dot(S)
            w = np.identity(len(X))
            teste(tipo,mist, S, X, curtose, w,A, save= save, white = do_white_fastica, path_to_csv = path_to_csv)
            teste(tipo,mist, S, X, negentropia, w, A,save = save, white = do_white_fastica, path_to_csv = path_to_csv)
            teste(tipo,mist, S, X, infomax, w, A, save = save, rate = 0.001, white=do_white_infomax, tipoinfomax = tipoinfomax, path_to_csv = path_to_csv)
            #teste(tipo = tipo,mist, S, X, infomax, winfomax, A,  save = save, rate = 0.001, white=do_white_infomax, tipoinfomax = 'sub', path_to_csv = path_to_csv)
            if n_sources == 2:
                Slist.append(s3)
            if n_sources == 3:
                Slist.append(s4)


In [20]:
def orgcsv(path):
    a = pd.read_csv(path)
    a = a.sort_values(by=['Algoritmo', 'Mistura'])
    a_edit = a.drop('Matriz de mistura', 1)
    a_edit = a_edit.drop('Matriz de separacao', 1)
    a_edit = a_edit.drop('Correlacao', 1)
    a_edit = a_edit.drop('MSE', 1)
    a_edit = a_edit.drop('White',1 )
    a_edit = a_edit.drop('Learning Rate',1 )
    return a_edit

In [21]:
def mist_mean_csv(path, df):
    algoritmos = df['Algoritmo'].unique()
    misturas = df['Mistura'].unique()
    for alg in algoritmos:
        for mst in misturas:
            sum_cor = sum_eqm = sum_amr = n_misturas =0
            for nm,(index, row) in enumerate(df.iterrows()):
                if row['Algoritmo'] == alg and row['Mistura'] == mst:
                    n_misturas += 1
                    sum_cor += abs(row['Correlacao Media'])
                    sum_eqm += row['Media EQM']
                    sum_amr += row['Indice de Amari']
                if ((row['Algoritmo'] != alg or row['Mistura'] != mst) and sum_cor != 0 and sum_eqm != 0 and sum_amr != 0) or (row['Algoritmo'] == alg and (nm+1) == len(df.index)):
                    corr_mistura = sum_cor/n_misturas
                    eqm_mistura = sum_eqm/n_misturas
                    amr_mistura = sum_amr/n_misturas
                    sum_cor = sum_eqm = sum_amr = 0
                    dic_mean = {'Tipo':row['Tipo'],'Algoritmo': alg, 'Mistura':mst, 
                                'Número de Fontes': row['Número de Fontes'], 'Indice de Amari media': amr_mistura,
                               'Correlacao Media': corr_mistura, 'EQM Media': eqm_mistura}
                    save_data_csv(dic_mean, create=True, path_to_csv = path)
                    break
    csv = pd.read_csv(path)
    return csv

In [22]:
def alg_mean_csv(path, df): #calcula a média de valor para cada algoritmo
    algoritmos = df['Algoritmo'].unique()
    data_dict = dict()
    for n_it, alg in enumerate(algoritmos):
        amr_list= []
        corr_list= []
        eqm_list = []
        sum_cor = sum_eqm = sum_amr = n_misturas = sum_sdr = sum_sir = sum_sar = 0
        n = df['Algoritmo'].value_counts().to_dict()
        n_alg = n[alg]
        nnn = n_it+1
        for nm,(index, row) in enumerate(df.iterrows()):
            if row['Algoritmo'] == alg:
                n_misturas += 1
                sum_cor += abs(row['Correlacao Media'])
                sum_eqm += row['EQM Media']
                sum_amr += row['Indice de Amari media']
                amr_list.append(abs(row['Correlacao Media']))
                corr_list.append(row['EQM Media'])
                eqm_list.append(row['Indice de Amari media'])
            if (row['Algoritmo'] != alg and sum_cor != 0 and sum_eqm != 0 and sum_amr != 0) or (row['Algoritmo'] == alg and n_alg == n_misturas and nnn == len(algoritmos)):
                corr_mistura = sum_cor/n_misturas
                eqm_mistura = sum_eqm/n_misturas
                amr_mistura = sum_amr/n_misturas
                eqm_array = np.array(eqm_list)
                amr_array = np.array(amr_list)
                corr_array = np.array(corr_list)
                alldata_alg = (eqm_array, amr_array, corr_array)
                std_corr = np.std(corr_list)
                std_eqm = np.std(eqm_list)
                std_amr = np.std(amr_list)
                dic_mean = {'Tipo':row['Tipo'],'Algoritmo': alg, 
                            'Número de Fontes': row['Número de Fontes'], 'Indice de Amari media': amr_mistura,
                           'Correlacao Media': corr_mistura, 'EQM Media': eqm_mistura, 'Indice de Amari std': std_amr
                           , 'EQM std': std_eqm, 'Correlacao std': std_corr}
                save_data_csv(dic_mean, create=True, path_to_csv = path)
                data_dict[alg] = alldata_alg
                sum_cor = sum_eqm = sum_amr = 0
                break 
    csv = pd.read_csv(path)
    csv['Indice de Amari std'] = np.std(amr_array)
    csv['EQM std'] = np.std(eqm_array)
    csv['Correlacao std'] = np.std(corr_array) #dict ={algoritmo: [eqm, amari, correlacao]}
    return csv, data_dict

In [23]:
def org_data(path_to_csv, path1 = None, path2 = None):
    df = orgcsv(path_to_csv)
    df_new = mist_mean_csv(path1, df)
    df_new2, data_dict = alg_mean_csv(path2, df_new)
    return df_new2, df_new, data_dict

In [44]:
dwinfomax = True
dwfastica = True
ptcsv = 'teste_nfontes_lap.csv'
save = True
tipo = 'laplace'
simula_gen_nfonts(n_gen = 50, n_amostra =1 ,tipo = tipo, do_white_fastica = dwfastica, do_white_infomax = dwinfomax, save =save, path_to_csv = ptcsv) 

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.15904530566213526

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.779243252767146

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.822143290169633

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 3.0515412290556365

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.6917483955560657

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 5.682973454167627

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 3.4341938021932314

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 4.492636936968095

Nome do algoritmo: Infomax super
O ín

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 8.779095628217013

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 7.136434000757664

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.6584758292055084

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.1281044984006552

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 1.8706267390389533

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 2.3563765253913784

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.1769892527720849

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.4803881623194393

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O í

O índice de Amari obtido para W é de 8.142911519485963

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 3.068852556150884

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 4.082353073693207

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 8.868844955166724

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.6549657815074919

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.3735652441963149

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 3.697881461326409

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 2.3139384492768986

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 6.719002540429698

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.994848823316437

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.0907265450687706

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 7.620708100830094

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.25963195178055054

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.6090568235829605

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.5876351326547489

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 2.302086277219206

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O ín

O índice de Amari obtido para W é de 8.112802541143822

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 6.026870903909005

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 6.482106750966613

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 11.029758452891283

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.16278452690174605

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.055888273911125275

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.8924208062527781

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.0870021110725787

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é 

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 5.332836886946003

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 8.5106169010335

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.14747303723148386

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.10053420782542544

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.8015128823999649

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 5.488769692641324

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 3.7646392443926247

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 7.4176593984767205

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O ín

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.9432085768379446

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 7.360732124397492

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.4008060776134639

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.27809629617953213

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.12216231199001792

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 4.289248825527108

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 3.7390374147523917

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 7.731311106507

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índ

In [45]:
def sep_data(path_to_read, path_to_save):
    df = pd.read_csv(path_to_read)
    algoritmos2 = df['Algoritmo'].unique()
    n_fontes = df['Número de Fontes'].unique()
    for alg2 in algoritmos2:
        for nf in n_fontes:
            df1 = df[['Algoritmo','Tipo', 'Mistura','Número de Fontes','Indice de Amari',
                     'Correlacao Media', 'Media EQM']]
            df1 = df1[df1.Algoritmo == alg2]
            df2 = df1[df1['Número de Fontes'] == nf]
            dic = {'Algoritmo': alg2, 'Numero de Fontes': nf, 
                   'Indice de Amari medio':np.mean(df2['Indice de Amari']),
                   'Correlacao media': np.mean(df2['Correlacao Media']),
                  'Media EQM': np.mean(df2['Media EQM'])}
            save_data_csv(dic, create=True, path_to_csv = path_to_save)
    csv = pd.read_csv(path_to_save)
    return csv

In [46]:
df = sep_data('teste_nfontes_lap.csv', 'lap_nfontes_org')
df

,Algoritmo,Numero de Fontes,Indice de Amari medio,Correlacao media,Media EQM
0,FastICA por Curtose,2,0.528639,0.980638,0.889964
1,FastICA por Curtose,3,2.367716,0.934985,0.829898
2,FastICA por Curtose,4,4.429109,0.915665,1.174513
3,FastICA por Negentropia,2,0.650300,0.970711,0.828179
4,FastICA por Negentropia,3,2.760019,0.913943,0.703563
5,FastICA por Negentropia,4,4.842600,0.899503,1.195745
6,Infomax super,2,1.628002,0.907688,0.995322
7,Infomax super,3,5.208966,0.822868,0.933271
8,Infomax super,4,9.399358,0.791031,1.389852


In [47]:
dwinfomax = True
dwfastica = True
ptcsv = 'teste_nfontes_uni.csv'
save = True
tipo = 'uniform'
simula_gen_nfonts(n_gen = 50, n_amostra =1 ,tipo = tipo, do_white_fastica = dwfastica, do_white_infomax = dwinfomax, save =save, path_to_csv = ptcsv) 

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.03569583905800067

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.06354968550620721

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.774693922970564

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.6530262739289536

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.1958822307472832

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 4.7859634908675455

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 7.67225132682395

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 4.688921222721782

Nome do algoritmo: Infomax sub
O índice 

Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 4.139806589080287

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 8.306474805722964

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 4.486633109604676

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 6.851741381019316

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 3.224666189368542

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 9.935192837350492

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.029285531232124518

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.02932806240776542

Nome do algoritmo: Infomax sub
O índice de Amari obtido para 

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 6.129020162534841

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 3.858808046761172

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 3.1671075763884073

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 9.281767883835677

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.6108063567557336

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.04109792899289655

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 3.1885863272022865

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.7147841245583708

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice 

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 8.911243170224374

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.05169722649329356

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.07007750723778816

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.29783075202842113

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 3.892830623166185

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.1348291332757867

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 6.447453118979653

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.5794781035327297

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índic

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 10.77285978696669

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.1829684331987187

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.19048111951103497

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.8511118725133742

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.8553756241469952

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.6956676479891943

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 3.1390297027728664

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 3.223876932574396

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 7.921487985618299

Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.05103621624629384

Número de iterações: 3
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.039712605688943414

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.1946426867960784

Número de fontes: 3
Número de iterações: 5
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.8508676524403367

Número de iterações: 4
Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.870748579089995

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 6.331495231382067

Número de fontes: 4
Número de iterações: 6
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.755533487592159

Número de iterações: 5
Nome do algoritmo: FastICA por Negentropia
O índice

O índice de Amari obtido para W é de 8.578357001659086



In [48]:
df = sep_data('teste_nfontes_uni.csv', 'uni_nfontes_org')
df

,Algoritmo,Numero de Fontes,Indice de Amari medio,Correlacao media,Media EQM
0,FastICA por Curtose,2,0.480827,0.979878,0.443521
1,FastICA por Curtose,3,1.818781,0.948465,0.983200
2,FastICA por Curtose,4,4.317756,0.911950,0.727293
3,FastICA por Negentropia,2,0.369508,0.986879,0.440898
4,FastICA por Negentropia,3,1.621823,0.957966,1.194906
5,FastICA por Negentropia,4,3.625181,0.931382,0.703332
6,Infomax sub,2,1.722758,0.902793,0.628830
7,Infomax sub,3,4.661548,0.850034,1.205707
8,Infomax sub,4,9.353825,0.795501,0.876169
